# 보스턴 집값 예측

## 패키지 로딩

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

import numpy as np
import pandas as pd


## 데이터 로드 및 확인

In [2]:
boston = pd.read_csv('datasets/HousingData.csv')
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [5]:
boston.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [4]:
boston = boston.fillna(method='ffill') # 'backfill'

## 독립변수, 종속변수 분할

In [6]:
X = boston.drop('MEDV', axis=1)
y = boston['MEDV']


## 학습/ 평가 데이터 분할

In [8]:
scaled_X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=0)

## 모델 생성 및 학습

In [9]:
# n_estimators= 의사결정 나무 개수
model = RandomForestRegressor(n_estimators=20, random_state=0)
model.fit(X_train, y_train)



RandomForestRegressor(n_estimators=20, random_state=0)

## 예측 및 평가

In [10]:
y_hat = model.predict(X_test)
print(f'MAE:{metrics.mean_absolute_error(y_test, y_hat):.3f}')
print(f'RMSE:{np.sqrt(metrics.mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 SCORE:{metrics.r2_score(y_test, y_hat):.3f}')

MAE:2.565
RMSE:4.149
R2 SCORE:0.793


## 하이퍼 파라미터 튜닝

In [21]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[3, 10, 20, 30, 40, 50],
          'max_features':[2, 4, 6, 8], # max_features : 의사결정 나무가 최적의 분할을 만들기 위해 사용되는 피처의 수
          'bootstrap':['False','True']
         }
model = RandomForestRegressor()

grid_cv = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error')
grid_cv.fit(scaled_X, y)

print(f'최적 파라미터 조합:{grid_cv.best_params_}')

최적 파라미터 조합:{'bootstrap': 'False', 'max_features': 6, 'n_estimators': 50}


In [23]:
y_hat = grid_cv.best_estimator_.predict(X_test)
print(f'MAE:{metrics.mean_absolute_error(y_test, y_hat):.3f}')
print(f'RMSE:{np.sqrt(metrics.mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 SCORE:{metrics.r2_score(y_test, y_hat):.3f}')

MAE:0.939
RMSE:1.501
R2 SCORE:0.973
